In [1]:
!pip install db-dtypes



  Using cached db_dtypes-1.0.4-py2.py3-none-any.whl (14 kB)


In [2]:
import pandas as pd
import numpy as np
import datetime
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [5]:
install("pymysql")

In [6]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='34.139.219.125',
                         user='root',
                         password='hp1811',
                         db='oltp')

# create cursor
cursor=connection.cursor()

In [7]:
credentials = service_account.Credentials.from_service_account_file('pax1-366517-7e41c87e2502.json')
client = bigquery.Client(credentials=credentials)

In [8]:
df_consumer_master=pd.read_sql(f'''select * from customer_master''',connection)
df_product_master=pd.read_sql(f'''select * from product_master''',connection)
df_order_details=pd.read_sql(f'''select * from order_details''',connection)
df_order_items=pd.read_sql(f'''select * from order_items''',connection)

In [9]:
df_order_items.head()

,orderid,productid,quantity


In [10]:
def insert_dim_order(df_order_details):
    g = df_order_details
    x = g[["orderid","order_status_update_timestamp","order_status"]]
    dim_order = x.reset_index()
    del dim_order["index"]
    return dim_order

In [11]:
def insert_dim_customer(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_customer =df_consumer_master[["customerid","name","address_id"]]
    dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
    dim_customer["end_date"] = np.nan
    dim_customer["end_date"]=pd.to_datetime(dim_customer["end_date"])
    return dim_customer

In [12]:
def insert_dim_address(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_address = df_consumer_master[["address_id","address","city","state","pincode"]]
    return dim_address

In [13]:
def insert_dim_product(df_product_master):
    dim_product = df_product_master
    dim_product["start_date"]=np.nan
    dim_product["start_date"]=pd.to_datetime(dim_product["start_date"])
    dim_product["end_date"]=np.nan
    dim_product["end_date"]=pd.to_datetime(dim_product["end_date"])
    return dim_product

In [14]:
def insert_f_order_details(df_order_details,df_order_items):
    f_order_details = pd.merge(df_order_details.groupby("orderid").tail(1), df_order_items, how='inner')[["orderid","order_status_update_timestamp","productid","quantity"]]
    f_order_details.columns = ["orderid","order_delivery_timestamp","productid","quantity"]
    return f_order_details

In [15]:
def insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items):
    x=df_order_details.groupby("orderid").head(1)[["customerid","orderid","order_status_update_timestamp"]].reset_index()
    del x["index"]
    y=df_order_details.groupby("orderid").tail(1)["order_status_update_timestamp"].reset_index()
    del y["index"]     
    fact_daily_orders = pd.concat([x,y],axis=1)
    fact_daily_orders.columns = ["customerid","orderid","order_received_timestamp","order_delivery_timestamp"]
    l=[]
    for i in fact_daily_orders["customerid"]:
             l.append(int(df_consumer_master.where(df_consumer_master["customerid"]==i).dropna()["pincode"]))
            # l is pincode column
    fact_daily_orders["pincode"] = l
        
    
    k=[]
    m=[]
    for i in df_order_items["productid"]:
        m=list(df_product_master.where(df_product_master["productid"]==i)["rate"].dropna() * df_order_items.iloc[i,2])
        k.append(m[0])
    df_order_items["Total"]=k    
    k = df_order_items.groupby('orderid')["Total"].sum()
    k1 = df_order_items.groupby('orderid')["quantity"].sum()
    k=k.tolist()
    fact_daily_orders["order_amount"]=k
    fact_daily_orders["item_count"]=list(k1)
    fact_daily_orders["order_delivery_time_seconds"] =fact_daily_orders["order_received_timestamp"]-fact_daily_orders["order_delivery_timestamp"] 
    return fact_daily_orders

In [16]:
dim_order = insert_dim_order(df_order_details)

In [17]:
dim_order.head()

,orderid,order_status_update_timestamp,order_status


In [18]:
dim_customer = insert_dim_customer(df_consumer_master)

In [19]:
dim_customer.head()

,customerid,name,address_id,start_date,end_date
0,1,Aaryahi Khurana,1,2022-08-23,NaT
1,2,Vaibhav Kade,2,2022-04-28,NaT
2,3,Trisha Yadav,3,2022-01-13,NaT
3,4,Romil Taneja,4,2022-06-23,NaT
4,5,Pihu Devan,5,2022-07-31,NaT


In [20]:
dim_address = insert_dim_address(df_consumer_master)

In [21]:
dim_product=insert_dim_product(df_product_master)

In [22]:
f_order_details = insert_f_order_details(df_order_details,df_order_items)


In [23]:
f_order_details = insert_f_order_details(df_order_details,df_order_items)


In [24]:
fact_daily_orders = insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items)

In [25]:
fact_daily_orders.head()

,customerid,orderid,order_received_timestamp,order_delivery_timestamp,pincode,order_amount,item_count,order_delivery_time_seconds


In [26]:
credentials = service_account.Credentials.from_service_account_file('pax1-366517-7e41c87e2502.json')

In [27]:
client = bigquery.Client(credentials=credentials)

In [28]:
tableRef1 = client.dataset("star_schema").table("dim_order")
client.load_table_from_dataframe(dim_order,tableRef1)

LoadJob<project=pax1-366517, location=US, id=646dabdc-f329-4447-bb97-94dc84bb0b33>

In [29]:
tableRef2 = client.dataset("star_schema").table("dim_customer")
client.load_table_from_dataframe(dim_customer,tableRef2)

LoadJob<project=pax1-366517, location=US, id=17c1edac-d666-4176-9538-70683b849319>

In [30]:
tableRef3 = client.dataset("star_schema").table("dim_address")
client.load_table_from_dataframe(dim_address,tableRef3)

LoadJob<project=pax1-366517, location=US, id=1bf9c55b-948f-4388-b511-b03ee9c4b2df>

In [31]:
tableRef4 = client.dataset("star_schema").table("dim_product")
client.load_table_from_dataframe(dim_product,tableRef4)

LoadJob<project=pax1-366517, location=US, id=714b2357-a15a-4c2a-b18b-c1e48c41991b>

In [32]:
tableRef5 = client.dataset("star_schema").table("f_order_details")
client.load_table_from_dataframe(f_order_details,tableRef5)

LoadJob<project=pax1-366517, location=US, id=dc03145f-313b-4be3-92d3-06e9d069b5cf>

In [33]:
tableRef6 = client.dataset("star_schema").table("fact_daily_orders")
client.load_table_from_dataframe(fact_daily_orders,tableRef6)

LoadJob<project=pax1-366517, location=US, id=1878d482-6ffd-4562-9978-e457e563b353>